# Computación evolutiva

Rama de la Inteligencia Artificial _inspirada_ en los mecanismos de la evolución biológica

- Las actuales especies y su comportamiento son **soluciones al problema de la vida**
  - Problema de la vida: Seguir vivos
- Parece que hemos llegado a soluciones complejas extremadamente adaptadas
  - La vida sigue existiendo tras [4250 millones de años](https://www.newscientist.com/article/dn14245-did-newborn-earth-harbour-life/) (década arriba o abajo)

¡Estos mecanismos son muy variados!

- [Colonias de hormigas](https://es.wikipedia.org/wiki/Algoritmo_de_la_colonia_de_hormigas), [enjambres de partículas](https://es.wikipedia.org/wiki/Optimizaci%C3%B3n_por_enjambre_de_part%C3%ADculas), [algoritmos genéticos](https://es.wikipedia.org/wiki/Algoritmo_gen%C3%A9tico), $\ldots$, cada uno con sus peculiaridades

**Nosotros nos centraremos en los algoritmos genéticos**

- Son **muy** potentes, y generalizándolos se agrupan muchos de estos mecanismos

## Algoritmos genéticos

Propuestos por John Holland, allá por 1975

Algoritmos iterativos _inspirados_ en la [Síntesis Evolutiva Moderna](https://es.wikipedia.org/wiki/S%C3%ADntesis_evolutiva_moderna), que integra, entre otros:

- _Selección natural_ del [darwinismo](https://es.wikipedia.org/wiki/Darwinismo), por la que los individuos más adaptados a su entorno tienen más probabilidad de sobrevivir
- _Herencia genética_ de la [genética mendeliana](https://es.wikipedia.org/wiki/Leyes_de_Mendel), el mecanismo que hace posible la transmisión de las características de progenitores a descendientes
- _Variación genética_ producida por diferentes tipos de [mutación](https://es.wikipedia.org/wiki/Mutación)

<center><img src='images/evolution.gif'></center>

## ¿Cuál es su estructura?

La estructura genérica de un algoritmo suele ser la siguiente
<br>
<center><img src='images/ga-schema.png'></center>
<br>

1. Se genera una población inicial
2. Se comprueba la condición de parada, es decir, si hemos llegado a alguna solución aceptable en la población
3. ¿Que sí? Paramos. ¿Que no? Pues repetimos los siguientes pasos una y otra vez hasta tener una población del tamaño deseado:
  1. Seleccionar $n$ individuos (progenitores)
  2. Recombinar (con cierta probabilidad) los $n$ individuos para obtener $m$ nuevos (progenie)
  3. Mutar (con cierta probabilidad) los $m$ individuoshasta completar su tamaño preestablecido
  4. Añadir los $m$ individuos en la nueva población
4. Reemplazar la antigua población por esta nueva y volver al paso 2

**Lo más normal es tanto $n$ como $m$ sea 2**, pero existen esquemas que usan valores diferentes

Vamos a ver es una introducción de cómo implementar cada una de las cajas sobre un problema clásico: la mochila de supervivencia

Antes, importaremos las librerías que se usarán a lo largo de la presentación

In [1]:
from random import choices, randint, random

## La mochila de supervivencia

Tenemos la idea de hacer una escapada al monte con nuestra nueva mochila de 20 litros

In [2]:
MAX_VOL = 20  # Máximo volumen de nuestra mochila

Como no cabe todo lo que nos gustaría llevar, por experiencia le hemos asignado unos "puntos de supervivencia" a cada elemento:

In [3]:
ELEMENTS_DATA = {
    'Navaja': (10, 1),
    'Yesca': (10, 1),
    'Bocata': (20, 5),
    'Agua': (15, 2),
    'Café': (3, 1),
    'Mosquetones': (2, 1),
    'Saco': (30, 6),
    'Tienda': (30, 9),
    'Cuerda': (10, 5),
    'Brújula': (30, 1),
    'Libro': (4, 3),
    'Tiritas': (10, 1),
    'Preservativos': (6, 3),
}  # Elemento -> (Puntos de superviviencia, Volumen)

Queremos encontrar por tanto, la combinación de elementos que:

- Más puntos de supervivencia me den: Cuanto más, mejor
- Mejor quepan en la mochila: Cuando menos espacio libre quede en la mochila, mejor


### Inicialización

Primero, necesitaremos una forma de representar a los individuos; podemos:

1. Ordenar los elementos alfabéticamente

In [4]:
ELEMENTS = sorted(ELEMENTS_DATA)
', '.join(ELEMENTS)

'Agua, Bocata, Brújula, Café, Cuerda, Libro, Mosquetones, Navaja, Preservativos, Saco, Tienda, Tiritas, Yesca'

2. Representar los genotipos como una lista de $1$'s y $0$'s, dependiendo de si está o no en la mochila

In [5]:
ALPHABET = 0, 1  # Los posibles valores de cada gen en el genotipo
print(f'Example genotype: {choices(ALPHABET, k=len(ELEMENTS))}')

Example genotype: [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0]


Segundo, definiremos un método para inicializar nuestra población de individuos

In [6]:
def initialize(pop_size, gen_size, alphabet):
    return [choices(alphabet, k=gen_size) for _ in range(pop_size)]

population = initialize(pop_size=5, gen_size=len(ELEMENTS), alphabet=ALPHABET)
for i, genotype in enumerate(population, 1):
    print(f'{i:02} -> {genotype}')

01 -> [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
02 -> [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0]
03 -> [1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
04 -> [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1]
05 -> [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0]


### Condición de parada

Determina si hemos llegado a una solución suficientemente buena

Usa generalmente la población y la generación actual para tomar la decisión

Nosotros implementaremos la condición de parada de **número de generaciones**

- Vamos, que cuando hemos hecho un número determinado de iteraciones paramos

In [7]:
def num_generations_stop(target_generation):
    def f(population, generation):
        return generation >= target_generation
    return f

Vemos si se comporta como esperamos, por ejemplo con un máximo de 100 generaciones

In [8]:
stop = num_generations_stop(100)
for generation in range(98, 102):
    print(f'Stop at {generation}?: {"yes" if stop(population, generation) else "no"}')

Stop at 98?: no
Stop at 99?: no
Stop at 100?: yes
Stop at 101?: yes


### Selección

Recordemos por dónde vamos:

<br>
<center><img src='images/ga-schema.png'></center>
<br>

Ahora viene un bucle donde estaremos seleccionando, cruzando y mutando individuos

- Todo ello para crear una nueva población con individuos descendientes de, probablemente, los mejores

La selección hará justamente eso, **seleccionar aleatoriamente individuos dándole más probabilidad a los mejores**:

- El individuo más apto es el que tiene más probabilidad de tener descendencia
- Las características que lo hacen el más apto estarán (presumiblemente) escritas en sus genes
  - ¡Y en la suerte! Que no se nos olvide
  
Existen muchos tipos diferentes de selección, nosotros utilizaremos la **selección por torneo**

#### Selección por torneo

1. Se determina un número $t$ de individuos
1. Por cada genotipo que queramos seleccionar
  1. Se toman $t$ genotipos aleatorios de la población
  2. El más apto de los $t$ individuos correspondientes a ese genotipo es elegido

... Espera, ¿y cómo determinamos cuál es el más apto?

De eso se encargará una función heurística que se denomina **aptitud** o _**fitness**_

- Devolverá un valor más grande cuando mayor sea su aptitud
- Es específico de nuestro problema; en nuestro caso habíamos quedado en que queríamos:
  - Cuantos más puntos de supervivencia, mejorMás puntos de supervivencia me den: Cuanto más, mejor
  - Cuanto menos espacio libre en la mochila, mejor

Bueno, pues programaremos primero una función `fitness` y luego una función `tournament` que la use

##### Fitness

Comprobará cómo de apto es un individuo; pero ojo, porque **individuo $\ne$ genotipo**

- Genotipo: Genes que posee un individuo
- Fenotipo: Expresión de esos genes **en el entorno**, es decir, el individuo

Primero determinaremos el fenotipo y luego calcularemos el fitness sobre este

In [9]:
def phenotype(genotype):
    return [elem for (i, elem) in zip(genotype, ELEMENTS) if i]

Y segundo, implementamos el fitness con las restricciones impuestas

In [10]:
def fitness(genotype):
    srv = sum(ELEMENTS_DATA[elem][0] for elem in phenotype(genotype))
    vol = sum(ELEMENTS_DATA[elem][1] for elem in phenotype(genotype))
    return srv - abs(MAX_VOL - vol) if vol <= MAX_VOL else 0

Veamos cómo se comportan:

In [11]:
for genotype in population:
    print(f'{genotype} (fitness = {fitness(genotype):03}): {phenotype(genotype)}')

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1] (fitness = 005): ['Mosquetones', 'Tiritas', 'Yesca']
[0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0] (fitness = 083): ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']
[1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1] (fitness = 000): ['Agua', 'Cuerda', 'Navaja', 'Preservativos', 'Saco', 'Tienda', 'Tiritas', 'Yesca']
[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1] (fitness = 063): ['Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas', 'Yesca']
[1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0] (fitness = 000): ['Agua', 'Bocata', 'Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']


##### Selección por torneo

Una vez tenemos el _fitness_, podemos aplicar cualquier función de selección, por ejemplo la de por torneo

In [12]:
def tournament_selection(t):
    def f(pop, n, fitness):
        return [max(choices(pop, k=t), key=fitness) for _ in range(n)]
    return f

Vamos a hacer una selección de 10 individuos sobre nuestra población

In [13]:
selection = tournament_selection(t=2)
genotypes = selection(population, n=10, fitness=fitness)
for genotype in genotypes:
    parent1, parent2 = selection(population, n=2, fitness=fitness)
    print(f'fitness = {fitness(genotype)}: {phenotype(genotype)}')

fitness = 83: ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']
fitness = 63: ['Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas', 'Yesca']
fitness = 83: ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']
fitness = 83: ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']
fitness = 63: ['Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas', 'Yesca']
fitness = 83: ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']
fitness = 0: ['Agua', 'Cuerda', 'Navaja', 'Preservativos', 'Saco', 'Tienda', 'Tiritas', 'Yesca']
fitness = 83: ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']
fitness = 63: ['Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas', 'Yesca']
fitness = 83: ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']


### Recombinación

La selección nos proporciona dos genotipos; ahora toca generar dos descendientes a partir de estos

- Para las listas tenemos un montón de operadores, nosotros vamos a implementar el que se conoce como _un punto de pivote_
| | | | |
| Pivot | 6 | | |
| --- | --- | --- | --- |
| Parent 1 | 💜💜💜💜💜💜💜💜💜💜 | Child 1 | 💜💜💜💜💜💜🧡🧡🧡🧡 |
| Parent 2 | 🧡🧡🧡🧡🧡🧡🧡🧡🧡🧡 | Child 2 | 🧡🧡🧡🧡🧡🧡💜💜💜💜 |

**Ojo**: La recombinación ocurre con cierta probabilidad, por lo que a veces los descendientes son los mismos progenitores

In [14]:
def one_pivot_recombination(parent1, parent2):
    pivot = randint(1, len(parent1) - 1)  # Punto de corte
    child1 = parent1[:pivot] + parent2[pivot:]
    child2 = parent2[:pivot] + parent1[pivot:]
    return child1, child2

Vamos a realizar una prueba de cruce:

In [15]:
genotypes = selection(population, n=10, fitness=fitness)
parent1, parent2 = selection(population, n=2, fitness=fitness)
print(f'Parent 1: {parent1} -> {phenotype(parent1)}')
print(f'Parent 2: {parent2} -> {phenotype(parent2)}')
child1, child2 = one_pivot_recombination(parent1, parent2)
print(f'Child 1:  {child1} -> {phenotype(child1)}')
print(f'Child 2:  {child2} -> {phenotype(child2)}')

Parent 1: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1] -> ['Mosquetones', 'Tiritas', 'Yesca']
Parent 2: [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0] -> ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas']
Child 1:  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0] -> ['Mosquetones', 'Tiritas']
Child 2:  [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1] -> ['Brújula', 'Cuerda', 'Libro', 'Mosquetones', 'Saco', 'Tiritas', 'Yesca']


### Mutación

Este operador es muy útil por dos razones principalmente:

- Permite alterar individuos existentes con cierta probabilidad de que mejores su aptitud
- Ayuda al la biodiversidad añadiendo información genética que podría no existir entre los individuos de la población

Debe garantizar que a la larga añade información genética nueva, pero sin producir pérdida de genes prometedores en individuos

Nuestra implementación será recorrer todos los genes del genotipo, con una probabilidad $p$ (pequeña) de que ese gen se reemplace

- Como nuestro alfabeto es binario, se reemplazará por el complementario

In [16]:
def binary_mutation(genotype, p):
    return [((g + 1) % 2) if random() < p else g for g in genotype]

Mutamos el genotipo del primer hijo resultado de la recombinación:

In [17]:
print(f'Child:   {child1} {phenotype(child1)}')
mutated = binary_mutation(child1, 1 / len(child1))
print(f'Mutated: {mutated} {phenotype(mutated)}')

Child:   [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0] ['Mosquetones', 'Tiritas']
Mutated: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0] ['Mosquetones', 'Tienda', 'Tiritas']


La probabilidad que hemos puesto es $p_0 = \frac{1}{len(g)}$, porque así todo individuo $g$ verá alterada, de media, un gen de su genotipo

### Reemplazo

Recordemos, de nuevo, por dónde vamos:

<br>
<center><img src='images/ga-schema.png'></center>
<br>

Este operador se encargará cambiar los genotipos de la población antigua por los de la nueva

Tradicionalmente existen dos aproximaciones, aunque en relidad existen casi tantas variantes como investigadores:

- _Steady-state_: Los dos individuos que se generan tras la selección-recombinación-mutación reemplazan a los peores
- _Generational_: Cuando tenemos una población de progenie igual de grande que la anterior la reemplazamos completamente

Nosotros implementaremos la _Generational_ clasica, es decir, reemplazar una población por otra

In [18]:
def generational_replacement(old_population, new_population):
    return new_population

### Algoritmo

Ya tenemos todos los operadores; primero definamos una serie de variables de configuración

In [19]:
NUM_GENERATIONS = 100  # Número de iteraciones del algoritmo
POPULATION_SIZE = 100  # Tamaño de la población
TOURNAMENT_SIZE = 3  # Tamaño de la muestra en el algoritmo de torneo

p_recombination = 0.9  # Probabilidad de cruce
p_mutation = 2. / len(ELEMENTS)  # Probabilidad de mutación

stop = num_generations_stop(NUM_GENERATIONS)
selection = tournament_selection(t=TOURNAMENT_SIZE)
recombination = one_pivot_recombination
mutation = binary_mutation
replacement = generational_replacement

Segundo, ejecutamos el algoritmo para ver qué solución nos arroja

In [20]:
# Inicializamos la población
population = initialize(POPULATION_SIZE, len(ELEMENTS), ALPHABET)

# Vamos iterando sobre generaciones hasta cumplir la condición de parada
generation = 0
while not stop(population, generation):
    # Creamos la nueva descendencia (en principio vacía)
    offspring = []
    # La rellenamos realizando operaciones de selección + cruce + mutación
    while len(offspring) < len(population):
        # Selección
        genotype1, genotype2 = selection(population, n=2, fitness=fitness)
        # Recombinación
        if random() < p_recombination:
            genotype1, genotype2 = recombination(genotype1, genotype2)
        # Mutación
        genotype1 = mutation(genotype1, p_mutation)
        genotype2 = mutation(genotype2, p_mutation)
        # Los añadimos a la descendencia
        offspring.append(genotype1)
        # Para no meter uno de más si el tamaño de la población es impar
        if len(offspring) < len(population):
            offspring.append(genotype2)
    
    # Reemplazamos la anterior población por la nueva
    population = replacement(population, offspring)
    # Incrementamos la generación
    generation += 1

# Al final del algoritmo, El mejor individuo de la población será la mejor
# solución encontrada al problema.
best_genome = max(population, key=fitness)

print(f'Fitness = {fitness(best_genome)}: {phenotype(best_genome)}')

Fitness = 125: ['Agua', 'Bocata', 'Brújula', 'Mosquetones', 'Navaja', 'Saco', 'Tiritas', 'Yesca']


## ¡Y hasta aquí!

Esto ha sido una introducción muy escueta y rápida a los algoritmos genéticos

Ahora, vamos a explorar un poco la implementación del juego de la vida para, de verdad, jugar con él